In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import HistGradientBoostingClassifier

In [15]:
original_train = pd.read_csv("train.csv").drop("Name", axis=1)
original_test = pd.read_csv("test.csv").drop("Name", axis=1)

train = original_train.copy()
test = original_test.copy()

le = LabelEncoder()
train["PassengerId"] = le.fit_transform(train["PassengerId"])
train["HomePlanet"] = le.fit_transform(train["HomePlanet"])
train["Cabin"] = le.fit_transform(train["Cabin"])
train["Destination"] = le.fit_transform(train["Destination"])

test["PassengerId"] = le.fit_transform(test["PassengerId"])
test["HomePlanet"] = le.fit_transform(test["HomePlanet"])
test["Cabin"] = le.fit_transform(test["Cabin"])
test["Destination"] = le.fit_transform(test["Destination"])
train.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0,1,False,149,2,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,1,0,False,2184,2,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,2,1,False,1,2,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,3,1,False,1,2,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,4,0,False,2186,2,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [5]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization


# Define the objective function to be maximized
def hgb_cv(learning_rate, max_depth, max_leaf_nodes, min_samples_leaf):
    # Define the HistGradientBoostingClassifier with the given hyperparameters
    clf = HistGradientBoostingClassifier(learning_rate=learning_rate,
                                          max_depth=int(max_depth),
                                          max_leaf_nodes=int(max_leaf_nodes),
                                          min_samples_leaf=int(min_samples_leaf),
                                          random_state=42)

    # Calculate cross-validation scores for the classifier
    scores = cross_val_score(clf, train.drop("Transported", axis=1), train["Transported"], cv=5, scoring='accuracy')

    # Return the mean cross-validation score
    return scores.mean()

# Define the hyperparameters and their respective ranges for Bayesian optimization
pbounds = {'learning_rate': (0.001, 0.1),
           'max_depth': (2, 10),
           'max_leaf_nodes': (2, 100),
           'min_samples_leaf': (1, 10)}

# Initialize the Bayesian optimizer with the objective function and hyperparameter ranges
optimizer = BayesianOptimization(f=hgb_cv, pbounds=pbounds, random_state=42)

# Run the Bayesian optimizer for 10 iterations
optimizer.maximize(init_points=5, n_iter=5)

# Print the optimal hyperparameters and corresponding score
print(optimizer.max)


|   iter    |  target   | learni... | max_depth | max_le... | min_sa... |
-------------------------------------------------------------------------
| 1         | 0.7168    | 0.03808   | 9.606     | 73.74     | 6.388     |
| 2         | 0.7789    | 0.01645   | 3.248     | 7.692     | 8.796     |
| 3         | 0.7279    | 0.06051   | 7.665     | 4.017     | 9.729     |
| 4         | 0.6838    | 0.08341   | 3.699     | 19.82     | 2.651     |
| 5         | 0.7208    | 0.03112   | 6.198     | 44.33     | 3.621     |
| 6         | 0.7316    | 0.07732   | 2.853     | 9.364     | 7.554     |
| 7         | 0.7856    | 0.01169   | 4.06      | 7.657     | 9.266     |
| 8         | 0.7763    | 0.01732   | 3.816     | 5.574     | 9.108     |
| 9         | 0.7206    | 0.04301   | 8.022     | 9.664     | 9.925     |
| 10        | 0.7663    | 0.001     | 5.018     | 6.308     | 6.681     |
{'target': 0.7855777330595537, 'params': {'learning_rate': 0.011693596212453231, 'max_depth': 4.059768835326959,

In [6]:

X = train.drop("Transported", axis=1)
y = train["Transported"]

X_train, X_test, y_train, y_test = train_test_split(X, y)



In [7]:
model = HistGradientBoostingClassifier(learning_rate=0.01169, max_depth=4.06, max_leaf_nodes=7.657, min_samples_leaf=9.266)
model.fit(X_train, y_train)

HistGradientBoostingClassifier(learning_rate=0.01169, max_depth=4.06,
                               max_leaf_nodes=7.657, min_samples_leaf=9.266)

In [8]:
y_pred = model.predict(X_test)

In [9]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[838 240]
 [253 843]]
              precision    recall  f1-score   support

       False       0.77      0.78      0.77      1078
        True       0.78      0.77      0.77      1096

    accuracy                           0.77      2174
   macro avg       0.77      0.77      0.77      2174
weighted avg       0.77      0.77      0.77      2174



In [10]:
y_pred = model.predict(test)

In [11]:
df_out = test["PassengerId"]
df_out = pd.DataFrame({
    "PassengerId": original_test["PassengerId"],
    "Transported": y_pred
    })
print(df_out.head())
df_out.to_csv("sample_submission.csv", index=False)

  PassengerId  Transported
0     0013_01        False
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01        False
